In [1]:
import random
from tqdm.auto import tqdm
from collections import defaultdict
import pandas as pd
from lambdag import LambdaG

/home/dniko/code/recursive_dict/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lambda_g = LambdaG()

Training reference models... 

Training reference models: 100%|██████████| 20/20 [00:00<00:00, 130055.94it/s]

Done.


In [3]:
records = []
with open("/mnt/d/datasets/Gungor_2018_VictorianAuthorAttribution_data-train.csv", "r", encoding="utf-8", 
          errors='replace') as f:
    for line in f:
        line = line.strip()
        if not line or '�' in line:
            continue
        records.append(line.split(","))
df = pd.DataFrame(records[1:], columns=records[0])

In [4]:
df.author.value_counts()[:10]

author
1     819
26    193
8     163
4     156
37    143
21    133
38    113
18     94
15     92
28     91
Name: count, dtype: int64

In [5]:
top_authors = df.author.value_counts()[:10].index.tolist()

In [6]:
author_to_fragments = {}
for author in top_authors:
    author_to_fragments[author] = df.loc[df.author == author].text.tolist()

In [7]:
results = defaultdict(dict)
for author in tqdm(author_to_fragments):
    target_author_fragments = author_to_fragments[author][:]
    random.shuffle(target_author_fragments)
    test_set_size = len(target_author_fragments) // 2
    train_fragments = target_author_fragments[test_set_size:]
    test_fragments = target_author_fragments[:test_set_size]
    lambda_g.train_known_author_model(train_fragments)
    results[author][author] = lambda_g.compute_lambda_g(test_fragments)
    for other_author in tqdm(top_authors, leave=False):
        if other_author == author:
            continue
        other_test_fragments = author_to_fragments[other_author]
        results[author][other_author] = lambda_g.compute_lambda_g(test_fragments)

  0%|          | 0/10 [04:31<?, ?it/s]


KeyboardInterrupt: 